***코드 제작: 2022113136 고태현***

***데이터 정보 및 특징 정리: 2020111318 김은채***

***발표: 2022113136 고태현, 2020111318 김은채, 2022111571 김태린***

Kaggle에서 "Alcohal Effects On Study"라는 raw data를 선택했습니다.

raw data에는 학년별 성적, 인구 통계, 사회 및 학교와 관련된 다양한 column들이 있었지만, 저희 조는 알코올이 학업에 유의미한 영향을 미치는지 확인하기 위해 필요가 없다고 생각한 데이터들은 삭제했습니다.

raw data는 Math.csv, Portuguese.csv로 총 2개의 csv 파일로 이뤄져 있었습니다.
두 data 모두 column은 모두 일치했기에, 필요한 column들만 추출하여 행 방향으로 병합했습니다.

저희가 추출한 data는 다음과 같습니다.

dalc(weekday_AC) -> 주중 음주량
* 범위: 1 ~ 5
* 숫자가 클수록 더 많은 음주량을 뜻함

walc(weekend_AC) -> 주말 음주량
* 범위: 1 ~ 5
* 숫자가 클수록 더 많은 음주량을 뜻함

studytime -> 한주 공부량
* 범위: 1 ~ 4
* 숫자가 클수록 더 많은 학습시간을 뜻함

failures -> FAIL 누적 횟수
* (횟수 = n) n이 0이상 3미만, 그대로 n으로 표시
* n이 3이상, 3으로 표시

sex -> 성별
* 'F' = female
* 'M' = male

age -> 나이
* 범위: 15 ~ 22


**Load data & preprocessing**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv("./drive/MyDrive/Maths.csv") # 음주가 수학 성적에 미치는 영향 - 데이터

# famrel, romantic, pastatus, famsize column 제거(필요없다고 판단)
df = df.drop(['famrel', 'Pstatus', 'famsize', 'romantic'], axis = 1)
df.dropna(inplace = True)

# 사용할 column들만 추출
df = df.loc[:, ['Dalc', 'Walc', 'studytime', 'failures', 'sex', 'age']]

# 가독성을 높이기 위해 column 이름 변경
df = df.rename(columns={'Dalc': 'weekday_AC', 'Walc' : 'weekend_AC'})

In [3]:
df_2 = pd.read_csv("./drive/MyDrive/Portuguese.csv") # 음주가 포르투갈어 성적에 미치는 영향 - 데이터

# famrel, romantic, pastatus, famsize column 제거(필요없다고 판단)
df_2 = df_2.drop(['famrel', 'Pstatus', 'famsize', 'romantic'], axis = 1)
df_2.dropna(inplace = True)

# 사용할 column들만 추출
df_2 = df_2.loc[:, ['Dalc', 'Walc', 'studytime', 'failures', 'sex', 'age']]

# 가독성을 높이기 위해 column 이름 변경
df_2 = df_2.rename(columns={'Dalc': 'weekday_AC', 'Walc' : 'weekend_AC'})

**데이터프레임 연결 및 누락값 제거**

In [4]:
# 행 방향으로 df, df_2 연결
df = pd.concat([df, df_2], axis=0)

In [5]:
df.dropna() #NaN(누락값) 제거

,weekday_AC,weekend_AC,studytime,failures,sex,age
0,1,1,2,0,F,18
1,1,1,2,0,F,17
2,2,3,2,3,F,15
3,1,1,3,0,F,15
4,1,2,2,0,F,16
...,...,...,...,...,...,...
644,1,2,3,1,F,19
645,1,1,2,0,F,18
646,1,1,2,0,F,18
647,3,4,1,0,M,17


**행/열의 수, 각 열의 기본 통계값 확인**

In [6]:
df.shape # 차원

(1044, 6)



*   총 1044개의 DATA
*   총 6가지의 attribute(column/dimension)



In [7]:
df.describe() #각 열의 기본 통계값

,weekday_AC,weekend_AC,studytime,failures,age
count,1044.000000,1044.000000,1044.000000,1044.000000,1044.000000
mean,1.494253,2.284483,1.970307,0.264368,16.726054
std,0.911714,1.285105,0.834353,0.656142,1.239975
min,1.000000,1.000000,1.000000,0.000000,15.000000
25%,1.000000,1.000000,1.000000,0.000000,16.000000
50%,1.000000,2.000000,2.000000,0.000000,17.000000
75%,2.000000,3.000000,2.000000,0.000000,18.000000
max,5.000000,5.000000,4.000000,3.000000,22.000000


**공분산/상관계수 구하기**

In [8]:
df.cov() # 공분산 구하기

,weekday_AC,weekend_AC,studytime,failures,age
weekday_AC,0.831223,0.735577,-0.121456,0.069594,0.150869
weekend_AC,0.735577,1.651494,-0.245619,0.090587,0.156627
studytime,-0.121456,-0.245619,0.696145,-0.083226,-0.008142
failures,0.069594,0.090587,-0.083226,0.430522,0.229731
age,0.150869,0.156627,-0.008142,0.229731,1.537537


In [9]:
df.corr() # 상관계수 구하기

,weekday_AC,weekend_AC,studytime,failures,age
weekday_AC,1.000000,0.627814,-0.159665,0.116336,0.133453
weekend_AC,0.627814,1.000000,-0.229073,0.107432,0.098291
studytime,-0.159665,-0.229073,1.000000,-0.152024,-0.007870
failures,0.116336,0.107432,-0.152024,1.000000,0.282364
age,0.133453,0.098291,-0.007870,0.282364,1.000000




1.   주중 음주량과 주말 음주량 -> 양의 상관관계
2.   주중 음주량과 학습시간 -> 음의 상관관계
3.   주중 음주량과 Fail 누적 수 -> 양의 상관관계
4.   주중 음주량과 나이 -> 양의 상관관계



**count 구하기**

In [10]:
df.count() # 각 column별 data의 개수 출력

weekday_AC    1044
weekend_AC    1044
studytime     1044
failures      1044
sex           1044
age           1044
dtype: int64

**studytime 기준으로 내림차순으로 정렬하기**

In [11]:
df.sort_values(by='studytime', ascending=False) # studytime이 가장 높은 순대로 정렬

,weekday_AC,weekend_AC,studytime,failures,sex,age
77,1,3,4,0,F,16
338,1,1,4,0,F,18
503,1,1,4,0,F,16
259,1,1,4,0,F,17
121,1,2,4,0,M,15
...,...,...,...,...,...,...
326,3,4,1,0,M,17
325,2,4,1,0,M,17
324,2,4,1,0,M,18
323,1,2,1,0,M,18


**열의 모든 원소에 함수 적용하여 df['sex']의 M,F를 각각 MALE, FEMALE로 변경**

In [12]:
df['sex'] = df['sex'].map({'M':'MALE', 'F':'FEMALE'})

**고유한 값 찾기**

In [13]:
# 각 column 별로 unique() function을 이용해 고유한 값 찾은 뒤 오름차순으로 정렬
print(sorted(df['weekday_AC'].unique()))
print(sorted(df['weekend_AC'].unique()))
print(sorted(df['studytime'].unique()))
print(sorted(df['failures'].unique()))
print(sorted(df['sex'].unique()))
print(sorted(df['age'].unique()))

[1, 2, 3, 4, 5]
[1, 2, 3, 4, 5]
[1, 2, 3, 4]
[0, 1, 2, 3]
['FEMALE', 'MALE']
[15, 16, 17, 18, 19, 20, 21, 22]


In [14]:
# 각 column별로 value_counts() function을 이용해 고유한 값들의 개수 출력
print(df['weekday_AC'].value_counts())
print(df['weekend_AC'].value_counts())
print(df['studytime'].value_counts())
print(df['failures'].value_counts())
print(df['sex'].value_counts())
print(df['age'].value_counts())

1    727
2    196
3     69
5     26
4     26
Name: weekday_AC, dtype: int64
1    398
2    235
3    200
4    138
5     73
Name: weekend_AC, dtype: int64
2    503
1    317
3    162
4     62
Name: studytime, dtype: int64
0    861
1    120
2     33
3     30
Name: failures, dtype: int64
FEMALE    591
MALE      453
Name: sex, dtype: int64
16    281
17    277
18    222
15    194
19     56
20      9
21      3
22      2
Name: age, dtype: int64


**누락된 값 다루기**

In [15]:
# 모든 column에서 누락된 값이 없음을 확인할 수 있다.
# OR 연산자를 활용해 행당 최소 하나의 열에서 누락값이 있을 경우 출력되도록 함
# 앞선 과정에서 이미 누락값을 모두 제외시켰기 때문에 이러한 결과가 나타남

df[df['weekday_AC'].isnull() | df['weekend_AC'].isnull() | df['studytime'].isnull() | df['failures'].isnull() | df['sex'].isnull() | df['age'].isnull()]

,weekday_AC,weekend_AC,studytime,failures,sex,age


**값에 따라 그룹핑**

In [16]:
# 'studytime' 열의 값으로 행을 그룹핑하고 평균 계산
# studytime과 주중 음주량, 주말 음주량, FAIL 수의 평균은 대체로 음의 상관관계임을 확인할 수 있음
# studytime과 나이의 평균은 대체로 양의 상관관계임을 확인할 수 있음

df.groupby('studytime').mean()

,weekday_AC,weekend_AC,failures,age
studytime,,,,
1,1.744479,2.706625,0.397476,16.690852
2,1.423459,2.212724,0.242545,16.737575
3,1.246914,1.833333,0.154321,16.913580
4,1.435484,1.887097,0.048387,16.322581


In [17]:
# 'sex', 'age' 열의 값으로 행을 그룹핑하고 평균 계산
# 나이와 주중 음주량, 주말 음주량의 평균은 대체로 음의 상관관계임을 확인할 수 있음
# 나이와 학습시간, FAIL 수의 평균은 대체로 양의 상관관계임을 확인할 수 있음

df.groupby(['sex', 'age']).mean()

weekday_AC  weekend_AC  studytime  failures
sex    age                                             
FEMALE 15     1.336842    1.873684   2.126316  0.210526
       16     1.220126    1.968553   2.025157  0.163522
       17     1.245614    2.035088   2.152047  0.105263
       18     1.236220    1.818898   2.267717  0.251969
       19     1.666667    2.030303   2.242424  0.969697
       20     1.200000    2.200000   2.200000  0.800000
       21     1.000000    1.000000   3.000000  2.000000
MALE   15     1.404040    2.050505   2.020202  0.060606
       16     1.655738    2.614754   1.672131  0.155738
       17     2.018868    3.188679   1.603774  0.424528
       18     1.989474    3.168421   1.789474  0.336842
       19     1.608696    2.130435   1.521739  1.043478
       20     2.000000    2.750000   1.250000  1.250000
       21     4.000000    2.500000   1.500000  2.500000
       22     5.000000    5.000000   1.000000  3.000000

**One-hot encoding**

In [18]:
# 성별 one-hot encoding
dummy = pd.get_dummies(df[['sex']], drop_first=True)
df_enc = pd.concat([df, dummy], axis=1)
df_enc
# 여성일 경우 sex_MALE = 0
# 남성일 경우 sex_MALE = 1

,weekday_AC,weekend_AC,studytime,failures,sex,age,sex_MALE
0,1,1,2,0,FEMALE,18,0
1,1,1,2,0,FEMALE,17,0
2,2,3,2,3,FEMALE,15,0
3,1,1,3,0,FEMALE,15,0
4,1,2,2,0,FEMALE,16,0
...,...,...,...,...,...,...,...
644,1,2,3,1,FEMALE,19,0
645,1,1,2,0,FEMALE,18,0
646,1,1,2,0,FEMALE,18,0
647,3,4,1,0,MALE,17,1


**distance와 cosine similarity를 구하기 위한 성별 column binary화**

In [20]:
df['sex'] = df['sex'].map({'MALE':0, 'FEMALE':1}) # distance 구하기 위해 성별을 0과 1로 변환 (남자: 0, 여자: 1)
# one-hot encoding 했던 걸 활용하면 되지만, 기존의 'sex' column을 제거하고, 
#'sex_MALE' column의 이름을 바꿔줘야한다는 번거로움이 있기에 활용하지 않았음.

**더 높은 정확도를 위한 Attribute Transform(normalization)**

In [21]:
normalization_df = (df - df.mean(axis=0))/df.std(axis=0)
normalization_df

,weekday_AC,weekend_AC,studytime,failures,sex,age
0,-0.542114,-0.999516,0.035589,-0.402913,0.875079,1.027397
1,-0.542114,-0.999516,0.035589,-0.402913,0.875079,0.220929
2,0.554721,0.556777,0.035589,4.169270,0.875079,-1.392007
3,-0.542114,-0.999516,1.234122,-0.402913,0.875079,-1.392007
4,-0.542114,-0.221369,0.035589,-0.402913,0.875079,-0.585539
...,...,...,...,...,...,...
644,-0.542114,-0.221369,1.234122,1.121148,0.875079,1.833865
645,-0.542114,-0.999516,0.035589,-0.402913,0.875079,1.027397
646,-0.542114,-0.999516,0.035589,-0.402913,0.875079,1.027397
647,1.651556,1.334924,-1.162945,-0.402913,-1.141659,0.220929


In [22]:
normalization_df = (df - df.min())/df.max()
df = normalization_df # 첫 번째 방법대신, 두 번째 방법으로 normalization 함.

**Minkowski Distance (0번째 샘플 ~ 12번째 샘플)**

In [23]:
np.linalg.norm(df.iloc[0] - df.iloc[12], ord=1) # 1차 거리

1.7863636363636362

In [24]:
np.linalg.norm(df.iloc[0] - df.iloc[12], ord=2) # 2차 거리

1.1140444521303063

In [25]:
np.linalg.norm(df.iloc[0] - df.iloc[12], ord=np.inf) # inf 거리

1.0

**(번외)첫 번째 사람과 가장 비슷한(거리가 가까운) 샘플은 몇 번째 사람일까?**

In [26]:
distance_vector = np.linalg.norm(df.iloc[0] - df.iloc[1:], axis = 1, ord = 1)
np.argmin(distance_vector) 

271

In [ ]:
df.iloc[0], df.iloc[271+1] # 272번째 인덱스에 있는 샘플이므로, 첫 번째 사람과 가장 비슷한 사람은 273번째 사람이다.

(weekday_AC    0.000000
 weekend_AC    0.000000
 studytime     0.250000
 failures      0.000000
 sex           1.000000
 age           0.136364
 Name: 0, dtype: float64, weekday_AC    0.000000
 weekend_AC    0.000000
 studytime     0.250000
 failures      0.000000
 sex           1.000000
 age           0.136364
 Name: 272, dtype: float64)

**Cosine similarity**

In [27]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(np.array(df.iloc[0]).reshape(1, -1), np.array(df.iloc[271+1]).reshape(1,-1))
# 앞서 첫 번째 사람과 가장 비슷했던 샘플인 272번째 인덱스에 있는 샘플과 
# 0번째 인덱스에 있는 샘플과의 코사인 유사도를 구해보니 1이 나오는 것을 확인할 수 있음
# 따라서 0번째 인덱스의 샘플과 272번째 인덱스의 샘플이 완전히 같다는 것을 알 수 있음

array([[1.]])

In [28]:
cosine_similarity(np.array(df.iloc[0]).reshape(1, -1), np.array(df.iloc[1023]).reshape(1,-1))
# 첫 번째 샘플과 맨 마지막 샘플간의 코사인 유사도를 구해보니 0.975~~가 나오는 것을 확인할 수 있음
# 두 샘플이 같진 않지만, 거의 비슷하다는 것을 확인할 수 있음

array([[0.97520082]])

**데이터 분석 결과 요약**

* 주중 음주량과 주말 음주량 -> 양의 상관관계
* 주중 음주량과 학습시간 -> 음의 상관관계
* 주중 음주량과 Fail 누적 수 -> 양의 상관관계
* 주중 음주량과 나이 -> 양의 상관관계

**최종 결과**

1. 음주량 ⇡, 학습시간 ⇣, Fail 수 ⇡, 나이 ⇡
2. 나이가 많을수록 대체로 음주량 증가
3. 음주는 대체로 학업에 악영향을 끼침

----------------------------------------------------
**(번외) 첫 번째 사람과 가장 유사한 사람은?**

* 첫 번째 사람과 가장 비슷한 사람은 273번째 사람(272번째 인덱스의 샘플)이다.
* 코사인 유사도가 1이므로, 0번째 인덱스의 샘플과 
272번째 인덱스의 샘플이 완전히 같다는 것을 알 수 있음



